In [1]:
using Pipe
using DataFrames
# using StatsPlots
using CSV
# using RollingFunctions
using Query
# using GLMakie

using JSON
using DataFrames
using Dates
using Colors
# using ColorSchemes
using Indicators

import Colors
using Plots
using Interact
using Statistics
# using JuliaDB
# using Wavelets,WaveletsExt
# using LinearAlgebra
# import Pkg; Pkg.add("ShiftedArrays")
# using ShiftedArrays
# include("utils.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-16020912613256990213\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-7685296078578583102\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-16020912613256990213\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
json_data_folder = "../user_data/data/binance"
json_filename = "BTC_USDT-1m.json"
json_filepath = "$json_data_folder/$json_filename"
f = open(json_filepath, "r")
json_data = JSON.parse(f)
close(f)

In [3]:
dfa = DataFrame(unix_time = Int64[], open=Float64[], high=Float64[], low=Float64[], close=Float64[], volume=Float64[])
for r in json_data push!(dfa, r) end
dfa.unix_time .= dfa.unix_time ./1000
# dfa.time .= unix2datetime.(dfa.unix_time)

insertcols!(dfa, :time => unix2datetime.(dfa.unix_time))
nrow(dfa)

129177

In [4]:
start_date = DateTime(2021, 3, 1)
end_date = DateTime(2021, 5, 21)
vdf = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]
ut, tt, hh, ll, cc = vdf.unix_time, vdf.time, vdf.high, vdf.low, vdf.close
# h14max = [fill(vdf[1,:high],8-1)...,rollmax(vdf.high, 8)...]
#  length(h14max), length(vdf.close)
# hlc = [hh ll cc]
length(tt)

116117

In [ ]:
ar = cc
ar_len = length(ar)
plot_box = Observable{Any}(dom"div"())


window_size = 60 * 8 * 1

num_windows = Int64(trunc(ar_len/window_size*4))

window_no_slider = slider(1:num_windows; label="start", default=7)

window_start = Interact.@map max(1, Int(&window_no_slider * window_size / 4))
window_end = Interact.@map min(ar_len, &window_start+window_size-1)

rng = Interact.@map &window_start: &window_end

put = Interact.@map ut[&rng]

# wdfo = Interact.@map dfo[(dfo.ut .>= first(&put)) .& (dfo.ut .<= last(&put)), :]

function draw_it_all(rng)
    put = ut[rng]
    wtdo = dfo[(dfo.ut .>= first(put)) .& (dfo.ut .<= last(put)), :]
    
    buys = wtdo[wtdo.reason .== :enter,:]
    losses = wtdo[(wtdo.reason .== :exit) && (wtdo.pnl .< 0),:]
    profits = wtdo[(wtdo.reason .== :exit) && (wtdo.pnl .> 0),:]
#     toolongs = dfo[(dfo.reason .== :toolong),:]
    
    f1 = plot(ut[rng], cc[rng], size=(1200,300),legend=nothing)
    plot!(put,ema50[rng])
#     plot!(put,ema100[rng])
#     plot!(put,ema150[rng])    
    
    plot!(put,ema50[rng]*0.995, fillrange=ema50[rng]*1.005, fillalpha=0.1)
    
    nrow(buys) > 0 && vline!(buys.ut, color=:blue)
    nrow(profits) > 0 && vline!(profits.ut, color=:green)
    nrow(losses) > 0 && vline!(losses.ut, color=:red)
    
    f2 = plot(put,rsi14[rng], linewidth=1, color=:blue)
    plot!(put,rsi26[rng], linewidth=1, color=:red)
    #     plot!(ut,rsi26)\n",
    hline!([30,70])
    plt = plot(f1, f2, layout=@layout[a{0.6h}; b{0.25h};], legend=nothing, size=(1200,400))
    dom"div"(vbox(plt,wtdo))
end

map!(draw_it_all, plot_box, rng)
ui = dom"div"(window_no_slider,plot_box)


In [ ]:
# x = [3, 1, 2];
# df = DataFrame(x=x)
df
r = last(df)
r.x = 22
df

In [ ]:
function backtest(e1, f1, e1ch1, e1ch2, far_1_dx2, sl, target_pe, expiry)
    capital = 10000.
    risk_percent = 0.02
    qfee = 0.001
    

    dfo = DataFrame(ut = Int64[], 
            order = Symbol[],
            oix = Int64[],
            reason = Symbol[],
            price = Float64[],
            target = Float64[],
            stop = Float64[],
            qty = Float64[],
            pnl = Float64[],
            capital = Float64[]
        )

    push!(dfo, (ut = ut[1], oix =0, order = :long, reason = :toolong, price = 0, stop = 0, target = 0,
                qty = 0, pnl = 0, capital = capital))


    in_long_pos = false 
    qlsellsig = 1.01
    qstop = 0.01
    in_i = 0
    for i in 200:length(cc)

        ord = last(dfo)
        if !in_long_pos
            if true && 
#                 far_1_dx2[i-1] < 0 && far_1_dx2[i] > 0
                cc[i] < f1[i] * (1 - e1ch1/100) &&
                cc[i] >  f1[i] * (1 - e1ch2/100) 
#                 cc[i] > pema_corr[i] * (1 - channel/100) && 
#                 cc[i-1] < pema_corr[i-1] * (1 - channel/100) 
#                     cc[i] < pema_corr[i] &&
#                     cc[i-1] > pema_corr[i-1] 


                stop_price = cc[i] * (1 - sl/100) # * (1-qfee)

                risk_size = capital * risk_percent

                buy_price = cc[i]*(1+qfee)

                risk_price_range = (buy_price - stop_price)
                buyq = risk_size / risk_price_range

                target = cc[i] * (1+target_pe/100)  # 2*risk_price_range

                push!(dfo, (ut = ut[i], oix=i, order = :long, reason = :enter, price = buy_price, qty = buyq,
                        target = target, stop = stop_price, pnl = 0, capital = capital))

                in_long_pos = true
            end
        else
#             last_min = minimum(cc[i-movesl:i-3])
            if cc[i] > f1[i] * (1+e1ch2/100)
                ord.stop = f1[i] * (1+e1ch2/100)
            elseif i - ord.oix >= expiry && cc[i] >= f1[i]
                ord.stop = cc[i] + 100
            elseif cc[i] >= ord.target
                ord.stop = cc[i]
            elseif cc[i-1] < f1[i-1] * (1 - e1ch2/100)
                ord.stop = cc[i]
            end
            
            if cc[i] <= ord.stop # SL
                sell_price = cc[i]*(1-qfee)
                profit_loss = ord.qty * (sell_price - ord.price)
                capital = capital + profit_loss

                push!(dfo, (ut = ut[i], oix=i, order = :long, reason = :exit, price = sell_price, stop = 0, 
                        target = 0, qty = ord.qty, pnl = profit_loss, capital = capital))

                in_long_pos = false
            end
           

        end
    #     println(capital)

    end
    CSV.write("trades.csv", dfo)
    dfo
end

In [ ]:
function smooth(ar,ema_1_sm_2_q, lag)
    nar = circshift(ar,lag)
    dar = ar - nar
    xar = ar + dar*ema_1_sm_2_q
end
function smooth2(ar, ema_1_sm_2_q, lag1, lag2)
    nar = smooth(ar,ema_1_sm_2_q,lag1)    
    xar = smooth(nar,ema_1_sm_2_q,lag2)
end
function der(ar, lag)
    nar = circshift(ar,lag)
    dar = ar - nar
end
function der2(ar, lag1, lag2)
    nar = der(ar,lag1)    
    xar = der(nar,lag2)
end
ar_len = length(cc)
plot_box = Observable{Any}(dom"div"())
res_box = Observable{Any}(dom"div"())
window_size = 60 * 8 * 1
num_windows = Int64(trunc(ar_len/window_size*4))
window_no_slider = slider(1:num_windows; label="start", default=7)
window_start = Interact.@map max(1, Int(&window_no_slider * window_size / 4))
window_end = Interact.@map min(ar_len, &window_start+window_size-1)
rng = Interact.@map &window_start: &window_end
put = Interact.@map ut[&rng]
ema_1_slider = slider(10:5:150; label="ema", default=50)
ema_1 = Interact.@map ema(cc, n=&ema_1_slider)
ema_1_dx_1_lag_slider = slider(0:100; label="e1dx1", default=14)
ema_1_dx_2_lag_slider = slider(0:100; label="e1dx2", default=14)

ema_1_ch_1_slider = slider(-5:0.05:5; label="ech1", default=0.5)
ema_1_ch_2_slider = slider(-5:0.05:5; label="ech2", default=0.5)
ema_1_sm_2_q = slider(-5:0.05:5; label="q1", default=0.5)

far_1 = Interact.@map smooth2(&ema_1,&ema_1_sm_2_q, &ema_1_dx_1_lag_slider,&ema_1_dx_2_lag_slider )
# far_1_der_2 = der2(&far_1,1,1)
sl_slider = slider(0:0.05:5; label="sl", default=1)
target_slider = slider(0:0.05:5; label="tgt", default=1)

far_1_dx2 = Interact.@map der2(&far_1,1,1)
# expire_slider = slider(1:200; label="exp", default=1)
# expire_slider[]=33
# movesl_slider = slider(1:50; label="movsl", default=1)
# movesl_slider[]=4

expiry_1_slider = slider(0:400; label="expiry", default=50)

dfo = Interact.@map backtest(&ema_1, &far_1, &ema_1_ch_1_slider, &ema_1_ch_2_slider, &far_1_dx2, 
    &sl_slider, &target_slider, &expiry_1_slider)

wdfo = Interact.@map filter(row -> (row.ut >= ut[&window_start] && row.ut <= ut[&window_end]), &dfo, view=false)
last2dfo = Interact.@map last(&dfo, 2)
# &dfo[(&dfo.ut .>= &window_start) .& (&dfo.ut .<= &window_end), :]

res_capital = Interact.@map trunc(last(&dfo).capital)

# losses = Interact.@map filter(row -> row.reason == :exit && row.pnl < 0, &dfo)
# profits = Interact.@map filter(row -> row.reason == :exit && row.pnl > 0, &dfo)
# no_losses = Interact.@map nrow(&losses)
# no_profits = Interact.@map nrow(&profits)
# tot_losses = Interact.@map sum((&losses).pnl)
# tot_profits = Interact.@map sum((&profits).pnl)

function draw_it_all(rng, ema_1, far_1, ema_1_ch_1, ema_1_ch_2, wdfo)
    put = ut[rng]   
    
    buys = wdfo[wdfo.reason .== :enter,:]
    losses = wdfo[(wdfo.reason .== :exit) .& (wdfo.pnl .< 0),:]
    profits = wdfo[(wdfo.reason .== :exit) .& (wdfo.pnl .> 0),:]
    
    f1 = plot(ut[rng], cc[rng], size=(1200,500))
    plot!(put,ema_1[rng], label="ema_1",lw=0.8)
    
    plot!(put,far_1[rng]*(1-ema_1_ch_1/100), fillrange=far_1[rng]*(1+ema_1_ch_1/100),
        lw=0,fillalpha=0.2, label="ch1")
    
    plot!(put,far_1[rng]*(1-ema_1_ch_2/100), fillrange=far_1[rng]*(1+ema_1_ch_2/100),
        lw=0,fillalpha=0.2, label="ch2")
# #     f1 = plot!(put,pema_lag[rng],label="lag ema",lw=1)
    f1 = plot!(put,far_1[rng], label="far_1",lw=1)
    
    nrow(buys) > 0 && vline!(buys.ut,lw=0.5, color=:blue)
    nrow(profits) > 0 && vline!(profits.ut,lw=0.5, color=:green)
    nrow(losses) > 0 && vline!(losses.ut,lw=0.5, color=:red)
    
    dom"div"(vbox(f1))
end

function draw_res(dfo)
    
    losses = dfo[(dfo.reason .== :exit) .& (dfo.pnl .< 0),:]
    profits = dfo[(dfo.reason .== :exit) .& (dfo.pnl .> 0),:]
    no_losses = nrow(losses)
    no_profits = nrow(profits)
    tot_losses = sum(losses.pnl)
    tot_profits = sum(profits.pnl)
    dom"div"(vbox((no_losses,no_profits),(trunc(tot_losses),trunc(tot_profits))))
end

map!(draw_res, res_box, dfo)

map!(draw_it_all, plot_box, rng, ema_1, far_1, ema_1_ch_1_slider, ema_1_ch_2_slider, wdfo)

ema_1_slider[]=120
ema_1_dx_1_lag_slider[] = 33
ema_1_dx_2_lag_slider[] = 10

ema_1_ch_1_slider[]=0.7
ema_1_ch_2_slider[]=0.85
ema_1_sm_2_q[]=1
sl_slider[]=2
expiry_1_slider[]=140
target_slider[] = 1.05
ui = dom"div"(hbox(
            vbox(ema_1_slider,ema_1_dx_1_lag_slider,ema_1_dx_2_lag_slider),
            vbox(ema_1_ch_1_slider,ema_1_ch_2_slider,ema_1_sm_2_q),
            vbox(sl_slider, res_capital, res_box),
            vbox(expiry_1_slider,target_slider)
        ),
        plot_box,window_no_slider)
# using Interact, Blink
# w = Window()
# body!(w, ui);
ui

In [ ]:
nrow(dfo[][(dfo[].reason .== :exit) .& (dfo[].pnl .< 0),:]),
    nrow(dfo[][(dfo[].reason .== :exit) .& (dfo[].pnl .> 0),:])

In [ ]:

sum([1,2,3] .- [3,4,5])

In [ ]:
function backtest(sl_slider, tp_slider, negated_slider, cooloff_slider, ema_1, ema_1_ch, ema_2, ema_2_ch)
#     println("$sl_slider, $lg_thresh_slider, $tp_slider")
    capital = 10000.
    risk_percent = 0.02
    qfee = 0.001
    

    dfo = DataFrame(ut = Int64[], 
            order = Symbol[],
            oix = Int64[],
            reason = Symbol[],
            price = Float64[],
#             target = Float64[],
            stop = Float64[],
            qty = Float64[],
            pnl = Float64[],
            capital = Float64[]
        )

    push!(dfo, (ut = ut[1], oix =0, order = :long, reason = :toolong, price = 0, stop = 0,
                qty = 0, pnl = 0, capital = capital))


    in_long_pos = false 
    qlsellsig = 1.01
    qstop = 0.01
    in_i = 0
    for i in 300:length(cc)

        ord = last(dfo)
        if !in_long_pos
            if true &&
                (ord.pnl > 0 || i - ord.oix > cooloff_slider) &&
                cc[i-1] < ema_2_ch[i-1] && cc[i] > ema_2_ch[i] 
#                 ema_1[i] < ema_2_ch[i] 
                
                stop_price = cc[i] * (1 - sl_slider/100)


#                 stop_price = cc[i] * (1 - sl/100) # * (1-qfee)

                risk_size = capital * risk_percent

                buy_price = cc[i]*(1+qfee)

                risk_price_range = (buy_price - stop_price)
                buyq = risk_size / risk_price_range

#                 target = cc[i] * (1+target_pe/100)  # 2*risk_price_range

                push!(dfo, (ut = ut[i], oix=i, order = :long, reason = :enter, price = buy_price, qty = buyq,
                        stop = stop_price, pnl = 0, capital = capital))

                in_long_pos = true
            end
        else
#             sell = false
            if cc[i] > ord.price * (1 + tp_slider/100)
                ord.stop = cc[i]
#                 sell = true
            elseif cc[i-1] > ema_2[i-1] && cc[i] < ema_2_ch[i] && i - ord.oix > negated_slider
                ord.stop = cc[i-1]
#             elseif i - ord.oix > negated_slider && mean(cc[i-negated_slider:i]) < ord.price
#                 ord.stop = cc[i-1]
            end
            
            if cc[i] <= ord.stop
                sell_price = ord.stop*(1-qfee)
                profit_loss = ord.qty * (sell_price - ord.price)
                capital = capital + profit_loss

                push!(dfo, (ut = ut[i], oix=i, order = :long, reason = :exit, price = sell_price, stop = 0, 
                         qty = ord.qty, pnl = profit_loss, capital = capital))

                in_long_pos = false
            end
           

        end
    #     println(capital)

    end
    CSV.write("trades.csv", dfo)
    dfo
end

In [ ]:
ar_len = length(cc)
plot_box = Observable{Any}(dom"div"())
res_box = Observable{Any}(dom"div"())
window_size = 60 * 8 * 3 * 3
num_windows = Int64(trunc(ar_len/window_size*4))
window_no_slider = slider(1:num_windows; label="start", default=7)
window_start = Interact.@map max(1, Int(&window_no_slider * window_size / 4))
window_end = Interact.@map min(ar_len, &window_start+window_size-1)
rng = Interact.@map &window_start: &window_end
put = Interact.@map ut[&rng]

ema_1_slider = slider(0:10:400; label="ema1", default=50)
ema_1 = Interact.@map sma(cc, n=&ema_1_slider)
ema_1_x_slider = slider(0:5:240; label="ema1X", default=0)
ema_1_y_slider = slider(-5:0.1:5; label="ema1Y", default=0)
ema_1_trans = Interact.@map circshift((&ema_1 .* (1 + &ema_1_y_slider/100)), &ema_1_x_slider)
ema_1_ch_slider = slider(-5:0.1:1; label="ema1ch", default=-1)
ema_1_ch = Interact.@map &ema_1_trans .* (1 + &ema_1_ch_slider / 100)

ema_2_slider = slider(0:10:400; label="ema2", default=50)
ema_2 = Interact.@map sma(cc, n=&ema_2_slider)
ema_2_x_slider = slider(0:5:240; label="ema2X", default=0)
ema_2_y_slider = slider(-5:0.1:5; label="ema2Y", default=0)
ema_2_trans = Interact.@map circshift((&ema_2 .* (1 + &ema_2_y_slider/100)), &ema_2_x_slider)
ema_2_ch_slider = slider(-5:0.1:1; label="ema2ch", default=-1)
ema_2_ch = Interact.@map &ema_2_trans .* (1 + &ema_2_ch_slider / 100)

negated_slider = slider(0:240; label="neg", default=-1)
cooloff_slider = slider(0:5:240; label="cool", default=-1)
# log_ema_slider = slider(0:100; label="emalog", default=7)
# log_val = Interact.@map log.(cc ./ &ema_1) .* 1000
# log_ema = Interact.@map ema(&log_val, n=&log_ema_slider)
# lg_thresh_slider = slider(-2:0.1:5; label="tsh", default=0.1)

sl_slider = slider(0:0.2:5; label="sl", default=0.1)
tp_slider = slider(0:0.1:5; label="tp", default=1)
# look_back_slider = slider(0:5:240; label="lb", default=10)


dfo = Interact.@map backtest(&sl_slider, &tp_slider, &negated_slider, &cooloff_slider, &ema_1_trans, &ema_1_ch, &ema_2_trans, &ema_2_ch)

wdfo = Interact.@map filter(row -> (row.ut >= ut[&window_start] && row.ut <= ut[&window_end]), &dfo, view=false)

# res_capital = Interact.@map trunc(last(&wdfo).capital)

buys = Interact.@map filter(row -> row.reason == :enter, &dfo)
losses = Interact.@map filter(row -> row.reason == :exit && row.pnl < 0, &dfo)
profits = Interact.@map filter(row -> row.reason == :exit && row.pnl > 0, &dfo)
no_losses = Interact.@map nrow(&losses)
no_profits = Interact.@map nrow(&profits)
tot_losses = Interact.@map Int(trunc(sum((&losses).pnl)))
tot_profits = Interact.@map Int(trunc(sum((&profits).pnl)))


wbuys = Interact.@map filter(row -> row.reason == :enter, &wdfo)
wlosses = Interact.@map filter(row -> row.reason == :exit && row.pnl < 0, &wdfo)
wprofits = Interact.@map filter(row -> row.reason == :exit && row.pnl > 0, &wdfo)

# , buys, profits, losses
function draw_it_all(rng, ema_1, ema_1_ch, ema_2, ema_2_ch, buys, profits, losses)
    put = ut[rng]   
    
    f1 = plot(put, cc[rng], size=(1200,500))
    plot!(put,ema_1[rng], label="ema1",lw=0.8)
    plot!(put,ema_2[rng], label="ema2",lw=0.8)
    
    plot!(put,ema_1[rng], fillrange=ema_1_ch[rng], fillalpha=0.15, label="ch1",lw=0)
    plot!(put,ema_2[rng], fillrange=ema_2_ch[rng], fillalpha=0.15, label="ch2",lw=0)
    
    
    nrow(buys) > 0 && vline!(buys.ut,lw=0.5, color=:blue)
    nrow(profits) > 0 && vline!(profits.ut,lw=0.5, color=:green)
    nrow(losses) > 0 && vline!(losses.ut,lw=0.5, color=:red)
    
#     f2 = plot(put, log_val[rng])
#     plot!(put, log_ema[rng])
#     hline!(-10,lw=1)
    
#     plt = plot(f1, f2, layout=@layout[a{0.6h}; b{0.25h};], size=(1400,500))
    dom"div"(vbox(f1))
end
# , wbuys, wprofits, wlosses
map!(draw_it_all, plot_box, rng, ema_1_trans, ema_1_ch, ema_2_trans, ema_2_ch, wbuys, wprofits, wlosses)

ema_1_slider[]=90
ema_2_slider[]=400
ema_1_ch_slider[]=-0.6
ema_2_ch_slider[]=-0.8
negated_slider[] = 75
cooloff_slider[] = 13
ema_1_x_slider[] = 0
ema_1_y_slider[] = 0
ema_2_x_slider[] = 0
ema_2_y_slider[] = 0

tp_slider[]=1.6
# log_ema_slider[] = 14
sl_slider[] = 2
# lg_thresh_slider[] = 1
# look_back_slider[] = 5
ui = dom"div"(hbox(
            vbox(ema_1_slider, ema_1_ch_slider, ema_1_x_slider, ema_1_y_slider),
            vbox(ema_2_slider, ema_2_ch_slider, ema_2_x_slider, ema_2_y_slider),
            vbox(sl_slider,tp_slider,negated_slider,cooloff_slider),
#         ,lg_thresh_slider,look_back_slider
#             vbox(log_ema_slider,sl_slider),
            Interact.@map vbox( (&no_losses,&no_profits),
                                    (&tot_losses,&tot_profits),
                                    (&tot_losses + &tot_profits))
        ),
        plot_box,window_no_slider)
# using Interact, Blink
# w = Window()
# body!(w, ui);
ui
# 1

In [ ]:
z = dfo[]
sum(z.pnl)

In [ ]:
last(dfo[])

In [ ]:
length(ar)

In [ ]:
# z = log_val[]
[(i,z[i]) for i in 1:length(z) if z[i] < -7]

In [ ]:
1.01 ^ 100